In [1]:
import os
import shutil

import pandas as pd
from salespy import SalesforceClass, EpicorClass

# login to Salesforce & get credentials
sf = SalesforceClass(
    username='bko@fs-elliott.com.test', sandbox='test',
    org_id='00D7A0000009Kmw'
)
creds = sf.login()
print(f'--- Logged Into Salesforce ---')

# epicor file directory
directory = (
    '/Users/beekayokay/OneDrive/Projects/'
    'Salesforce Epicor Integration/E10 Reports To Upload'
)

# loop through epicor files and merge
count = 0
epicor = EpicorClass()
bookings_df = pd.DataFrame()
for each in os.listdir(directory):
    temp_df = pd.DataFrame()
    if each == '.DS_Store':
        continue
    try:
        temp_df = epicor.bookings_to_df(os.path.join(directory, each))
        bookings_df = bookings_df.append(temp_df, ignore_index=True)
    except IsADirectoryError:
        continue
print(f'--- Merged Files: {bookings_df.size} ---')

# aggregate df by OrderNum & OrderLn
bookings_df = epicor.aggregate_df(bookings_df)
print(f'--- Aggregated Data: {bookings_df.size} ---')

--- Logged Into Salesforce ---
--- Merged Files: 1261577 ---
--- Aggregated Data: 1333241 ---


In [4]:
from datetime import datetime
book_df = bookings_df['Book_Date__c']
book_df = book_df.append(bookings_df['Original_Book_Date__c'])
book_df.drop_duplicates(inplace=True)
book_list = [datetime.strftime(each, '%Y-%m-%d') for each in list(book_df)]
book_query = ', '.join(list(book_list))

In [5]:
query_call = (
    'SELECT Actual_Date__c, Fiscal_Month__c '
    'FROM Fiscal_Date__c '
    'WHERE Actual_Date__c IN '
    f"({book_query})"
)

fisc_df = sf.bulk_query(creds=creds, object_api='Fiscal_Date__c', query_call=query_call)

In [6]:
cur_fisc_df = fisc_df.copy()
org_fisc_df = fisc_df.copy()
cur_fisc_df.rename(columns={'Actual_Date__c': 'Book_Date__c'}, inplace=True)
org_fisc_df.rename(columns={'Actual_Date__c': 'Original_Book_Date__c', 'Fiscal_Month__c': 'Original_Fiscal_Month__c'}, inplace=True)

In [7]:
cur_fisc_df['Book_Date__c'] = pd.to_datetime(cur_fisc_df['Book_Date__c'])
cur_fisc_df['Fiscal_Month__c'] = pd.to_datetime(cur_fisc_df['Fiscal_Month__c'])
org_fisc_df['Original_Book_Date__c'] = pd.to_datetime(org_fisc_df['Original_Book_Date__c'])
org_fisc_df['Original_Fiscal_Month__c'] = pd.to_datetime(org_fisc_df['Original_Fiscal_Month__c'])

In [9]:
bookings_df = bookings_df.merge(org_fisc_df, how='left', on='Original_Book_Date__c')
bookings_df = bookings_df.merge(cur_fisc_df, how='left', on='Book_Date__c')